In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
import os
import seaborn as sns
from scipy import stats
from madule.simulation import *
from madule.plots import *
from statsmodels.stats.anova import AnovaRM

## 1. Figures

> Plots of choice responses for each individuals

In [ ]:

# List of subjects
subList = ['sub-004', 'sub-012', 'sub-020', 'sub-025', 'sub-026', 'sub-029', 'sub-030',
           'sub-033', 'sub-034', 'sub-036', 'sub-040', 'sub-041', 'sub-042', 'sub-045',
           'sub-047', 'sub-048', 'sub-052', 'sub-054', 'sub-056', 'sub-059', 'sub-060',
           'sub-064', 'sub-065', 'sub-067', 'sub-069', 'sub-070', 'sub-071', 'sub-074',
           'sub-075', 'sub-076', 'sub-077', 'sub-078', 'sub-079', 'sub-080', 'sub-081',
           'sub-082', 'sub-083', 'sub-085', 'sub-087', 'sub-088', 'sub-089', 'sub-090',
           'sub-092', 'sub-108', 'sub-109']


for subName in subList:
    
    # List of existing .csv files for each session and run realted to the subject
    files = ['/ses-02achieva7t/' + subName + '_ses-02achieva7t_task-DA_run-1_beh.csv',
             '/ses-02achieva7t/' + subName + '_ses-02achieva7t_task-DA_run-2_beh.csv',
             '/ses-03achieva7t/' + subName + '_ses-03achieva7t_task-DA_run-1_beh.csv',
             '/ses-03achieva7t/' + subName + '_ses-03achieva7t_task-DA_run-2_beh.csv']

    # Main directory of the subject
    subMainDirec = '/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/originalfMRIbehFiles/'
    
    # Making empty Dataframe to be concatenated in all four .csv file of the subject
    data = pd.DataFrame([])
    for i in range(len(files)):
        dirc = subMainDirec + subName + files[i]
        df = pd.read_csv(dirc)
        data = pd.concat([data, df])

    #save file name
    saveFile = subMainDirec + subName + '/' + subName + '_behData'
    plotChosenCorrect(data = data, subName = subName, saveFile = saveFile)

## Pooling data

In [2]:
# List of subjects
subList = ['sub-004', 'sub-012',            'sub-025', 'sub-026', 'sub-029', 'sub-030',
           'sub-033', 'sub-034', 'sub-036', 'sub-040', 'sub-041', 'sub-042', 'sub-045',
           'sub-047', 'sub-048', 'sub-052', 'sub-054', 'sub-056', 'sub-059', 'sub-060',
           'sub-064', 'sub-065', 'sub-067', 'sub-069', 'sub-070', 'sub-071', 'sub-074',
           'sub-075', 'sub-076', 'sub-077', 'sub-078', 'sub-079', 'sub-080', 'sub-081',
           'sub-082', 'sub-083', 'sub-085', 'sub-087', 'sub-088', 'sub-089', 'sub-090',
           'sub-092', 'sub-108', 'sub-109']
# Group level
randomGroupLabel = pd.read_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/randomGroupLabel.csv')
# concatenating data
dataAll = pd.DataFrame([])
for subName in subList:
    
    # List of existing .csv files for each session and run realted to the subject
    files = ['/ses-02achieva7t/' + subName + '_ses-02achieva7t_task-DA_run-1_beh.csv',
             '/ses-02achieva7t/' + subName + '_ses-02achieva7t_task-DA_run-2_beh.csv',
             '/ses-03achieva7t/' + subName + '_ses-03achieva7t_task-DA_run-1_beh.csv',
             '/ses-03achieva7t/' + subName + '_ses-03achieva7t_task-DA_run-2_beh.csv']

    # Main directory of the subject
    subMainDirec = '/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/originalfMRIbehFiles/'
    
    # Making empty Dataframe to be concatenated in all four .csv file of the subject
    data = pd.DataFrame([])
    for i in range(len(files)):
        dirc = subMainDirec + subName + files[i]
        df = pd.read_csv(dirc)
        data = pd.concat([data, df])
    # set the name of participants to each data
    data['sub_ID'] = subName
    # read and then set the number of group
    labelSes1 = int(randomGroupLabel.loc[randomGroupLabel['sub-ID'] == subName, 'ses-02'])
    labelSes2 = int(randomGroupLabel.loc[randomGroupLabel['sub-ID'] == subName, 'ses-03'])
    data.loc[data['session'] == 1, 'group'] = str(labelSes1)
    data.loc[data['session'] == 2, 'group'] = str(labelSes2)
    # Detection of irregular responces (no-responses or error responces)
    temp = data['pushed'].to_numpy().astype(int)
    dataClear = data[temp>=0]
    #save file name
    saveFile = subMainDirec + subName + '/' + subName + '_behData'
    # Concatenating each data
    dataAll = pd.concat([dataAll, dataClear])

# Save concatenated data over all particiapnts
dataAll.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/behAll.csv', index=False)

## Choice Proportion and RT in Group 2

In [13]:
# Save concatenated data over group lable 2
dataAllGroup2 = dataAll.loc[dataAll['group']=='2']
dataAllGroup2.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/dataAllGroup2.csv', index=False)

In [98]:
# Repeated mesure ANOVA in choice proportion
aovrm2way = AnovaRM(data = dataAllGroup2, depvar='correctChoice', subject = 'sub_ID',
                    within=['session','block'], aggregate_func='mean')
res2way = aovrm2way.fit()
res2way.summary()

ValueError: Data is unbalanced.

In [40]:
# Repeated mesure ANOVA in responce time
aovrm2way = AnovaRM(data = dataAllGroup2, depvar='RT', subject = 'sub_ID',
                    within=['session','block'], aggregate_func='mean')
res2way = aovrm2way.fit()
res2way.summary()

,F Value,Num DF,Den DF,Pr > F
session,6.3961,1.0000,22.0000,0.0191
block,0.2349,1.0000,22.0000,0.6327
session:block,0.1212,1.0000,22.0000,0.7310


## Choice Proportion and RT in Groups 1 and 3

In [12]:
## Choice Proportion and RT in Group 1 and 3
dataAllGroup13 = dataAll.loc[(dataAll['group']=='1') | (dataAll['group']=='3')]
dataAllGroup13.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/dataAllGroup13.csv', index=False)


In [88]:
# Repeated mesure ANOVA across session in choice proportion
aovrm2way = AnovaRM(data = dataAllGroup13, depvar='correctChoice', subject = 'sub_ID',
                    within=['session','block'], aggregate_func='mean')
res2way = aovrm2way.fit()
res2way.summary()

,F Value,Num DF,Den DF,Pr > F
session,1.3674,1.0000,20.0000,0.2560
block,3.2493,1.0000,20.0000,0.0865
session:block,0.0684,1.0000,20.0000,0.7963


In [89]:
# Repeated mesure ANOVA across session in choice proportion
aovrm2way = AnovaRM(data = dataAllGroup13, depvar='correctChoice', subject = 'sub_ID',
                    within=['session','block'], aggregate_func='mean')
res2way = aovrm2way.fit()
res2way.summary()

,F Value,Num DF,Den DF,Pr > F
session,1.3674,1.0000,20.0000,0.2560
block,3.2493,1.0000,20.0000,0.0865
session:block,0.0684,1.0000,20.0000,0.7963


In [90]:
# Repeated mesure ANOVA across session in response time
aovrm2way = AnovaRM(data = dataAllGroup13, depvar='RT', subject = 'sub_ID',
                    within=['session','block'], aggregate_func='mean')
res2way = aovrm2way.fit()
res2way.summary()

,F Value,Num DF,Den DF,Pr > F
session,23.6925,1.0000,20.0000,0.0001
block,0.3493,1.0000,20.0000,0.5611
session:block,2.5279,1.0000,20.0000,0.1275


In [91]:
# Repeated mesure ANOVA across group in choice proportion
aovrm2way = AnovaRM(data = dataAllGroup13, depvar='correctChoice', subject = 'sub_ID',
                    within=['group','block'], aggregate_func='mean')
res2way = aovrm2way.fit()
res2way.summary()

,F Value,Num DF,Den DF,Pr > F
group,2.6827,1.0000,20.0000,0.1171
block,3.2493,1.0000,20.0000,0.0865
group:block,1.5619,1.0000,20.0000,0.2258


In [92]:
# Repeated mesure ANOVA across group in response time
aovrm2way = AnovaRM(data = dataAllGroup13, depvar='RT', subject = 'sub_ID',
                    within=['group','block'], aggregate_func='mean')
res2way = aovrm2way.fit()
res2way.summary()

,F Value,Num DF,Den DF,Pr > F
group,0.0052,1.0000,20.0000,0.9430
block,0.3493,1.0000,20.0000,0.5611
group:block,4.7595,1.0000,20.0000,0.0412


In [93]:
dataAllGroup13

,session,run,stimActFirst,block,stimActBlock,trialNumber,stimOnset,yellowOnLeftSide,leftCanBePushed,winAmtLeft,...,decFBjitOnset,FBOnset,yellowCorrect,pushCorrect,correctChoice,wonAmount,totalAmount,ITIOnset,sub_ID,group
1,1,1,Act,Act,1,3,12.808,1,1,6,...,15.754,22.754,1,1,1.0,6,6,23.958,sub-004,1
4,1,1,Act,Act,1,6,49.864,0,1,71,...,53.131,57.062,1,0,1.0,29,35,58.264,sub-004,1
5,1,1,Act,Act,1,7,61.545,0,1,15,...,64.721,68.362,1,0,1.0,85,120,69.567,sub-004,1
6,1,1,Act,Act,1,8,73.445,1,0,68,...,76.722,81.679,0,1,0.0,0,120,82.887,sub-004,1
8,1,1,Act,Act,1,10,99.551,1,0,22,...,102.317,108.004,0,1,1.0,78,198,109.211,sub-004,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2,2,Act,Act,2,80,942.687,1,0,88,...,945.258,951.676,0,1,1.0,12,2098,952.893,sub-109,3
79,2,2,Act,Act,2,81,959.096,0,1,7,...,961.856,964.044,0,1,1.0,7,2105,965.260,sub-109,3
81,2,2,Act,Act,2,83,987.428,1,0,28,...,990.062,993.712,1,0,0.0,0,2105,994.929,sub-109,3
82,2,2,Act,Act,2,84,997.423,0,0,53,...,1000.882,1007.897,1,1,1.0,47,2152,1009.113,sub-109,3


In [41]:
# Average over choice responce and RT across participants in group lable 2
choiceRTGroup2Mean = dataAllGroup2.groupby(['sub_ID', 'session', 'block'], as_index=False)['correctChoice', 'RT'].mean()
#choiceRTGroup2Mean.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/choiceRTGroup2Mean.csv', index=False)
# Gran average over choice responce and RT across participants in group lable 2
choiceRTGroup2Grand = choiceRTGroup2Mean.groupby(['session', 'block'], as_index=False)['correctChoice', 'RT'].mean()
#choiceRTGroup2Grand.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/choiceRTGroup2Grand.csv', index=False)
# Grand Standard Error over choice responce and RT across participants in group lable 2
nParts2 = len(np.unique(choiceRTGroup2Mean['sub_ID']))
choiceRTGroup2SE = choiceRTGroup2Mean.groupby(['session', 'block'], as_index=False).apply(lambda x:x[['correctChoice', 'RT']].std()/nParts2)
#choiceRTGroup2SE.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/choiceRTGroup2SE.csv', index=False)

/tmp/ipykernel_89564/2129569399.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  choiceRTGroup2Mean = dataAllGroup2.groupby(['sub_ID', 'session', 'block'], as_index=False)['correctChoice', 'RT'].mean()
/tmp/ipykernel_89564/2129569399.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  choiceRTGroup2Grand = choiceRTGroup2Mean.groupby(['session', 'block'], as_index=False)['correctChoice', 'RT'].mean()


In [154]:



# Save concatenated data over group lables 1 and 3
dataAllGroup13 = dataAll.loc[(dataAll['group']=='1') | (dataAll['group']=='3')]
dataAllGroup13.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/dataAllGroup13.csv', index=False)

# save choice proportion and RT measurements for group labels 1 and 2
dataAllGroup12Ses1 = dataAll.loc[((dataAll['group']=='1') & (dataAll['session']==1)) | ((dataAll['group']=='2') & (dataAll['session']==1))]
dataAllGroup12Ses1.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/dataAllGroup12Ses1.csv', index=False)

# save choice proportion and RT measurements for group labels 1 and 2
dataAllGroup32Ses1 = dataAll.loc[((dataAll['group']=='3') & (dataAll['session']==1)) | ((dataAll['group']=='2') & (dataAll['session']==1))]
dataAllGroup32Ses1.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/dataAllGroup32Ses1.csv', index=False)


23

In [101]:
dataAllGroup2.groupby(['session', 'block'], as_index=False)['correctChoice', 'RT']

/tmp/ipykernel_3349111/1913149991.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dataAllGroup2.groupby(['session', 'block'], as_index=False)['correctChoice', 'RT']


In [97]:
dataAllGroup13GrandAvr = dataAllGroup13.groupby(['session', 'block'], as_index=False)['correctChoice', 'RT'].mean()
dataAllGroup13.groupby(['group', 'block'], as_index=False)['correctChoice', 'RT'].mean()

/tmp/ipykernel_3349111/1162384504.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dataAllGroup13GrandAvr = dataAllGroup13.groupby(['session', 'block'], as_index=False)['correctChoice', 'RT'].mean()
/tmp/ipykernel_3349111/1162384504.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dataAllGroup13.groupby(['group', 'block'], as_index=False)['correctChoice', 'RT'].mean()


,group,block,correctChoice,RT
0,1,Act,0.501200,1.564469
1,1,Stim,0.502130,1.505003
2,3,Act,0.495152,1.522167
3,3,Stim,0.475529,1.545862


> Treatment effect over Choice Propotion and between ON and OFF Medication group

In [83]:

# calculating choise propotion and averaging RT across independent varibales
choiceRT = dataAll.groupby(['group', 'session', 'block'], as_index=False)['correctChoice', 'RT'].mean()
# save choice proportion and RT measurements for all groups labels
choiceRT.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/choiceRT.csv', index=False)
# save choice proportion and RT measurements for group labels 1 and 2
choiceRTGroup13 = choiceRT.loc[(choiceRT['group']==1) | (choiceRT['group']==3)]
choiceRTGroup13.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/choiceRTGroup13.csv', index=False)
# save choice proportion and RT measurements for all group label 2
choiceRTGroup2 = choiceRT.loc[choiceRT['group']==2]
choiceRTGroup2.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/choiceRTGroup2.csv', index=False)


/tmp/ipykernel_3349111/3939150282.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  choiceRT = dataAll.groupby(['group', 'session', 'block'], as_index=False)['correctChoice', 'RT'].mean()


In [78]:
dataAllGroup2.groupby(['session', 'block', 'trialNumber'], as_index=False)['correctChoice', 'RT'].mean()

/tmp/ipykernel_3349111/70541274.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dataAllGroup2.groupby(['session', 'block', 'trialNumber'], as_index=False)['correctChoice', 'RT'].mean()


,session,block,trialNumber,correctChoice,RT
0,1,Act,2,0.500000,2.057071
1,1,Act,3,0.350000,1.710700
2,1,Act,4,0.476190,1.576619
3,1,Act,5,0.652174,1.519478
4,1,Act,6,0.500000,1.531636
...,...,...,...,...,...
331,2,Stim,81,0.625000,1.488542
332,2,Stim,82,0.363636,1.653318
333,2,Stim,83,0.571429,1.513000
334,2,Stim,84,0.318182,1.391818


/tmp/ipykernel_3349111/390003240.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  choiceRTGroup13.groupby(['session', 'block'], as_index=False)['correctChoice', 'RT'].mean()


,session,block,correctChoice,RT
0,1,Act,0.503112,1.620349
1,1,Stim,0.495541,1.570349
2,2,Act,0.493667,1.468310
3,2,Stim,0.481701,1.484859


In [80]:
choiceRTGroup13

,sub-ID,group,session,block,correctChoice,RT
0,sub-004,1,1,Act,0.466667,1.555547
1,sub-004,1,1,Stim,0.506849,1.469370
2,sub-004,3,2,Act,0.512821,1.363936
3,sub-004,3,2,Stim,0.413333,1.559467
8,sub-025,1,2,Act,0.526316,1.388566
...,...,...,...,...,...,...
171,sub-108,3,2,Stim,0.400000,1.405507
172,sub-109,1,1,Act,0.573171,1.329268
173,sub-109,1,1,Stim,0.540541,1.118365
174,sub-109,3,2,Act,0.569620,1.330949


In [70]:
choiceRTGroup13.groupby(['group', 'block'], as_index=False)['correctChoice', 'RT'].mean()

/tmp/ipykernel_3349111/1372695140.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  choiceRTGroup13.groupby(['group', 'block'], as_index=False)['correctChoice', 'RT'].mean()


,group,block,correctChoice,RT
0,1,Act,0.501239,1.564799
1,1,Stim,0.501600,1.504523
2,3,Act,0.495540,1.523859
3,3,Stim,0.475642,1.550686


In [71]:
choiceRTGroup2.groupby(['session', 'block'], as_index=False)['correctChoice', 'RT'].mean()

/tmp/ipykernel_3349111/1282709228.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  choiceRTGroup2.groupby(['session', 'block'], as_index=False)['correctChoice', 'RT'].mean()


,session,block,correctChoice,RT
0,1,Act,0.497148,1.554141
1,1,Stim,0.498179,1.547042
2,2,Act,0.482203,1.450367
3,2,Stim,0.496145,1.431694
